In [ ]:
# Section I: Explore the Data

In [ ]:
# 1. Products Data (PRODUCTS_TAKEHOME.csv)

In [ ]:
# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load Products Data
products = pd.read_csv('PRODUCTS_TAKEHOME.csv')

# Function to check missing data
def missing_summary(df, name):
    print(f"Missing data for {name}:\n{df.isnull().sum()}\n")

# Products Data Exploration
missing_summary(products, "Products")

# Check top manufacturers
print("Top 10 Manufacturers:\n", products['MANUFACTURER'].value_counts().head(10))

# Check top categories
print("Top Categories:\n", products['CATEGORY_1'].value_counts())

# Barcode Distribution - Outlier Check
plt.figure(figsize=(10, 5))
sns.boxplot(x=products['BARCODE'])
plt.title("Product Barcode Distribution (Outlier Check)")
plt.show()

In [ ]:
# 2. Transactions Data (TRANSACTION_TAKEHOME.csv)

In [ ]:
# Import libraries
import pandas as pd

# Load Transactions Data
transactions = pd.read_csv('TRANSACTION_TAKEHOME.csv')

# Function to check missing data
def missing_summary(df, name):
    print(f"Missing data for {name}:\n{df.isnull().sum()}\n")

# Transactions Data Exploration
missing_summary(transactions, "Transactions")

# Check final quantity counts (including data cleaning need)
print("Final Quantity Counts:\n", transactions['FINAL_QUANTITY'].value_counts())

# Review top Final Sale values
print("Top Final Sale Values:\n", transactions['FINAL_SALE'].value_counts().head(10))

In [ ]:
# 3. Users Data (USER_TAKEHOME.csv)

In [ ]:
# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load Users Data
users = pd.read_csv('USER_TAKEHOME.csv')

# Function to check missing data
def missing_summary(df, name):
    print(f"Missing data for {name}:\n{df.isnull().sum()}\n")

# Users Data Exploration
missing_summary(users, "Users")

# Check most common birth dates (check for placeholder values like 1970-01-01)
print("Most Common Birth Dates:\n", users['BIRTH_DATE'].value_counts().head(10))

# Check unusual gender values
print("Gender Distribution:\n", users['GENDER'].value_counts())

# Check language distribution
print("Language Distribution:\n", users['LANGUAGE'].value_counts())

# Analyze birth year distribution for potential age segmentation
users['BIRTH_YEAR'] = pd.to_datetime(users['BIRTH_DATE'], errors='coerce').dt.year

plt.figure(figsize=(12, 6))
sns.histplot(users['BIRTH_YEAR'].dropna(), bins=30, kde=True)
plt.title("User Birth Year Distribution")
plt.show()

In [ ]:
# Data Cleaning

In [ ]:
# 1. Products Data:

In [ ]:
import pandas as pd

# Load data
products = pd.read_csv('PRODUCTS_TAKEHOME.csv')

# Make a working copy to preserve the original
products_clean = products.copy()

# Fill missing categories with 'Unknown' - these are optional but help with categorization completeness
products_clean['CATEGORY_3'] = products_clean['CATEGORY_3'].fillna('Unknown')
products_clean['CATEGORY_4'] = products_clean['CATEGORY_4'].fillna('Unknown')

# Standardize missing manufacturer and brand - for consistency in grouping and filtering
products_clean['MANUFACTURER'] = products_clean['MANUFACTURER'].fillna('Unknown Manufacturer')
products_clean['BRAND'] = products_clean['BRAND'].fillna('Unknown Brand')

# Replace 'PLACEHOLDER MANUFACTURER' with 'Unknown Manufacturer' to remove placeholder noise
products_clean['MANUFACTURER'] = products_clean['MANUFACTURER'].replace('PLACEHOLDER MANUFACTURER', 'Unknown Manufacturer')

# Drop products with missing barcodes - these cannot be linked to transactions
products_clean = products_clean.dropna(subset=['BARCODE'])

# Identify barcode length to check for unrealistic (too long) barcodes
products_clean['BARCODE_LENGTH'] = products_clean['BARCODE'].astype(str).apply(len)

# Flag products with barcodes longer than 15 digits as potential outliers
products_clean['BARCODE_FLAG'] = products_clean['BARCODE_LENGTH'].apply(lambda x: 'OUTLIER' if x > 15 else 'OK')

# Drop the temporary BARCODE_LENGTH column - no longer needed
products_clean = products_clean.drop(columns=['BARCODE_LENGTH'])

# Summary statistics
original_count = len(products)
cleaned_count = len(products_clean)
placeholder_count = (products['MANUFACTURER'] == 'PLACEHOLDER MANUFACTURER').sum()
unknown_manufacturer_count = (products_clean['MANUFACTURER'] == 'Unknown Manufacturer').sum()
barcode_outlier_count = (products_clean['BARCODE_FLAG'] == 'OUTLIER').sum()

# Create a cleanup summary dictionary
cleanup_summary = {
    'Original Product Count': original_count,
    'After Cleanup Product Count': cleaned_count,
    'Dropped Products (Missing Barcode)': original_count - cleaned_count,
    'Replaced PLACEHOLDER MANUFACTURER': placeholder_count,
    'Total Unknown Manufacturers': unknown_manufacturer_count,
    'Flagged Barcode Outliers': barcode_outlier_count
}

# Display the summary
print("Products Cleanup Summary:")
for k, v in cleanup_summary.items():
    print(f"{k}: {v}")

In [ ]:
# 2. Transactions Data:

In [ ]:
import pandas as pd

# Load transactions dataset
transactions = pd.read_csv('TRANSACTION_TAKEHOME.csv')

# Make a working copy to preserve the original
transactions_clean = transactions.copy()

# Convert FINAL_QUANTITY to numeric, replacing 'zero' with 0
transactions_clean['FINAL_QUANTITY'] = transactions_clean['FINAL_QUANTITY'].replace('zero', 0).astype(float)

# Identify transactions with missing barcodes - these may be non-scanned items (like produce or services)
transactions_clean['MISSING_BARCODE'] = transactions_clean['BARCODE'].isnull()

# Create a flag for transactions with missing or blank FINAL_SALE
transactions_clean['FINAL_SALE_FLAG'] = transactions_clean['FINAL_SALE'].apply(lambda x: 'MISSING' if pd.isnull(x) or x.strip() == '' else 'OK')

# Convert dates to datetime format to enable proper analysis
transactions_clean['PURCHASE_DATE'] = pd.to_datetime(transactions_clean['PURCHASE_DATE'], errors='coerce')
transactions_clean['SCAN_DATE'] = pd.to_datetime(transactions_clean['SCAN_DATE'], errors='coerce')

# Summary Statistics
original_count = len(transactions)
missing_barcode_count = transactions_clean['MISSING_BARCODE'].sum()
missing_sale_count = (transactions_clean['FINAL_SALE_FLAG'] == 'MISSING').sum()

# Create a cleanup summary dictionary
cleanup_summary = {
    'Original Transaction Count': original_count,
    'Transactions with Missing Barcodes': missing_barcode_count,
    'Transactions with Missing or Blank Final Sale': missing_sale_count,
}

# Display the summary
print("Transactions Cleanup Summary:")
for k, v in cleanup_summary.items():
    print(f"{k}: {v}")

In [ ]:
# 3. Users Data:

In [ ]:
import pandas as pd

# Load users dataset
users = pd.read_csv('USER_TAKEHOME.csv')

# Create a working copy to preserve the original data
users_clean = users.copy()

# Convert BIRTH_DATE to datetime; handle errors and leave invalid dates as NaT (missing)
users_clean['BIRTH_DATE'] = pd.to_datetime(users_clean['BIRTH_DATE'], errors='coerce')

# Fill missing STATE, LANGUAGE, and GENDER with placeholders
users_clean['STATE'] = users_clean['STATE'].fillna('Unknown State')
users_clean['LANGUAGE'] = users_clean['LANGUAGE'].fillna('unknown')
users_clean['GENDER'] = users_clean['GENDER'].fillna('unknown')

# Standardize GENDER field to a controlled list
gender_map = {
    'female': 'female',
    'male': 'male',
    'transgender': 'other',
    'non_binary': 'other',
    'Non-Binary': 'other',
    'prefer_not_to_say': 'prefer_not_to_say',
    'Prefer not to say': 'prefer_not_to_say',
    'not_listed': 'other',
    'not_specified': 'unknown',
    'My gender isn\'t listed': 'other',
    'unknown': 'unknown'
}
users_clean['GENDER'] = users_clean['GENDER'].map(gender_map)

# Flag placeholder birth dates (assumed placeholder date is 1970-01-01)
users_clean['PLACEHOLDER_BIRTHDATE'] = users_clean['BIRTH_DATE'].apply(
    lambda x: 'PLACEHOLDER' if x == pd.Timestamp('1970-01-01') else 'OK'
)

# Extract birth year for future age-based segmentation
users_clean['BIRTH_YEAR'] = users_clean['BIRTH_DATE'].dt.year

# Summary Statistics and Findings
original_count = len(users)
missing_birth_date_count = users['BIRTH_DATE'].isnull().sum()
missing_state_count = users['STATE'].isnull().sum()
missing_language_count = users['LANGUAGE'].isnull().sum()
missing_gender_count = users['GENDER'].isnull().sum()
placeholder_birthdate_count = (users_clean['PLACEHOLDER_BIRTHDATE'] == 'PLACEHOLDER').sum()

# Show Final Normalized Gender Distribution
normalized_gender_counts = users_clean['GENDER'].value_counts()

# Print Final Cleanup Summary
print("Users Cleanup Summary:")
print(f"Original User Count: {original_count}")
print(f"Missing Birth Dates (original): {missing_birth_date_count}")
print(f"Missing States (original): {missing_state_count}")
print(f"Missing Languages (original): {missing_language_count}")
print(f"Missing Genders (original): {missing_gender_count}")
print(f"Placeholder Birth Dates (1970-01-01): {placeholder_birthdate_count}")
print("\nNormalized Gender Distribution (After Cleanup):")
print(normalized_gender_counts)

In [ ]:
# Section II: Provide SQL Queries

In [ ]:
# SQLite Setup:

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data into SQLite In-Memory Database
conn = sqlite3.connect(":memory:")

# Load cleaned data into SQLite tables
products_cleaned = pd.read_csv('PRODUCTS_TAKEHOME_CLEANED.csv')
transactions_cleaned = pd.read_csv('TRANSACTION_TAKEHOME_CLEANED.csv')
users_cleaned = pd.read_csv('USER_TAKEHOME_CLEANED.csv')

products_cleaned.to_sql('products', conn, index=False, if_exists='replace')
transactions_cleaned.to_sql('transactions', conn, index=False, if_exists='replace')
users_cleaned.to_sql('users', conn, index=False, if_exists='replace')

print("Tables loaded into SQLite:")
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))

In [ ]:
# Closed-Ended Questions:

In [ ]:
# 1. What are the top 5 brands by receipts scanned among users 21 and over?

In [ ]:
# Top 5 Brands by Receipts Scanned for Users 21+
query_1 = '''
SELECT p.BRAND, COUNT(DISTINCT t.RECEIPT_ID) AS receipt_count
FROM transactions t
JOIN products p ON t.BARCODE = p.BARCODE
JOIN users u ON t.USER_ID = u.ID
WHERE (strftime('%Y', 'now') - u.BIRTH_YEAR) >= 21
GROUP BY p.BRAND
ORDER BY receipt_count DESC
LIMIT 5;
'''
top_brands = pd.read_sql(query_1, conn)

print("\nTop 5 Brands by Receipts Scanned (21+):")
print(top_brands)

# Horizontal Bar Chart
plt.figure(figsize=(10, 6))
sns.barplot(y='BRAND', x='receipt_count', data=top_brands)
plt.title('Top 5 Brands by Receipts Scanned (21+)')
plt.show()

In [ ]:
# 2. What are the top 5 brands by sales among users that have had their account for at least six months?

In [ ]:
# Top 5 Brands by Sales for Users with 6+ Months Account Age
query_2 = '''
SELECT p.BRAND, SUM(CAST(t.FINAL_SALE AS FLOAT)) AS total_sales
FROM transactions t
JOIN products p ON t.BARCODE = p.BARCODE
JOIN users u ON t.USER_ID = u.ID
WHERE date('now') >= date(u.CREATED_DATE, '+6 months')
AND t.FINAL_SALE_FLAG = 'OK'
GROUP BY p.BRAND
ORDER BY total_sales DESC
LIMIT 5;
'''
top_sales_brands = pd.read_sql(query_2, conn)

print("\nTop 5 Brands by Sales (6+ Months Users):")
print(top_sales_brands)

# Pie Chart
plt.figure(figsize=(8, 8))
plt.pie(top_sales_brands['total_sales'], labels=top_sales_brands['BRAND'], autopct='%1.1f%%', startangle=140)
plt.title('Top 5 Brands by Sales (6+ Months Users)')
plt.show()

In [ ]:
# 3. What is the percentage of sales in the Health & Wellness category by generation?

In [ ]:
# Percentage of Health & Wellness Sales by Generation
query_3 = '''
WITH user_gen AS (
    SELECT 
        ID,
        CASE 
            WHEN BIRTH_YEAR >= 1997 THEN 'Gen Z'
            WHEN BIRTH_YEAR BETWEEN 1981 AND 1996 THEN 'Millennials'
            WHEN BIRTH_YEAR BETWEEN 1965 AND 1980 THEN 'Gen X'
            ELSE 'Baby Boomers'
        END AS generation
    FROM users
)
, category_sales AS (
    SELECT 
        g.generation,
        SUM(CAST(t.FINAL_SALE AS FLOAT)) AS health_wellness_sales
    FROM transactions t
    JOIN products p ON t.BARCODE = p.BARCODE
    JOIN user_gen g ON t.USER_ID = g.ID
    WHERE p.CATEGORY_1 = 'Health & Wellness'
    AND t.FINAL_SALE_FLAG = 'OK'
    GROUP BY g.generation
)
, total_sales AS (
    SELECT SUM(CAST(t.FINAL_SALE AS FLOAT)) AS total_sales
    FROM transactions t
    WHERE t.FINAL_SALE_FLAG = 'OK'
)
SELECT 
    cs.generation,
    (cs.health_wellness_sales * 100.0 / ts.total_sales) AS health_wellness_percentage
FROM category_sales cs, total_sales ts
ORDER BY health_wellness_percentage DESC;
'''
gen_health_wellness = pd.read_sql(query_3, conn)

print("\nHealth & Wellness Sales by Generation:")
print(gen_health_wellness)

# Vertical Bar Chart
plt.figure(figsize=(8, 5))
sns.barplot(x='generation', y='health_wellness_percentage', data=gen_health_wellness)
plt.title('Health & Wellness Sales by Generation')
plt.show()

In [ ]:
# Open-Ended Questions:

In [ ]:
# 1. Who are Fetch’s power users?

In [ ]:
# Fetch Power Users
query_4 = '''
SELECT 
    u.ID AS user_id,
    COUNT(DISTINCT t.RECEIPT_ID) AS total_receipts,
    SUM(t.FINAL_QUANTITY) AS total_items_purchased,
    SUM(CAST(t.FINAL_SALE AS FLOAT)) AS total_spent
FROM transactions t
JOIN users u ON t.USER_ID = u.ID
WHERE t.FINAL_SALE_FLAG = 'OK'
GROUP BY u.ID
ORDER BY total_receipts DESC, total_spent DESC, total_items_purchased DESC
LIMIT 10;
'''
power_users = pd.read_sql(query_4, conn)

print("\nFetch Power Users:")
print(power_users)

# Scatter Plot - Total Receipts vs Total Spent
plt.figure(figsize=(8, 5))
plt.scatter(power_users['total_receipts'], power_users['total_spent'], color='green')
plt.title('Top 10 Power Users - Receipts vs Spend')
plt.xlabel('Total Receipts')
plt.ylabel('Total Spent')
plt.grid(True)
plt.show()

In [ ]:
# 2. Which is the leading brand in the Dips & Salsa category?

In [ ]:
# Leading Brand in Dips & Salsa
query_5 = '''
SELECT p.BRAND, SUM(CAST(t.FINAL_SALE AS FLOAT)) AS total_sales
FROM transactions t
JOIN products p ON t.BARCODE = p.BARCODE
WHERE LOWER(p.CATEGORY_3) LIKE '%dip%' OR LOWER(p.CATEGORY_3) LIKE '%salsa%'
AND t.FINAL_SALE_FLAG = 'OK'
GROUP BY p.BRAND
ORDER BY total_sales DESC
LIMIT 1;
'''
top_dips_brand = pd.read_sql(query_5, conn)

print("\nLeading Brand in Dips & Salsa:")
print(top_dips_brand)

# Simple Text Output
print(f"The leading brand in Dips & Salsa is: {top_dips_brand['BRAND'][0]} with sales of ${top_dips_brand['total_sales'][0]:,.2f}")

In [ ]:
# 3. At what percent has Fetch grown year over year?

In [ ]:
# Year-over-Year Growth (Based on User Account Creation Date)

# Ensure dates in users table are correctly formatted in SQLite (done during initial load)
query_6 = '''
WITH yearly_users AS (
    SELECT 
        strftime('%Y', CREATED_DATE) AS year,
        COUNT(*) AS user_count
    FROM users
    WHERE CREATED_DATE IS NOT NULL
    GROUP BY year
)
, yoy_growth AS (
    SELECT 
        year,
        user_count,
        LAG(user_count) OVER (ORDER BY year) AS previous_year_user_count,
        CASE 
            WHEN LAG(user_count) OVER (ORDER BY year) IS NOT NULL
            THEN (user_count - LAG(user_count) OVER (ORDER BY year)) * 100.0 / LAG(user_count) OVER (ORDER BY year)
            ELSE NULL
        END AS yoy_growth_percent
    FROM yearly_users
)
SELECT * FROM yoy_growth;
'''

yoy_growth = pd.read_sql(query_6, conn)

print("\nYear-over-Year User Growth (Based on Account Creation Date):")
print(yoy_growth)

# Line Chart for YoY Growth
plt.figure(figsize=(10, 6))
plt.plot(yoy_growth['year'], yoy_growth['yoy_growth_percent'], marker='o', linestyle='-', color='purple')
plt.title('Year-over-Year User Growth (Based on Account Creation Date)')
plt.xlabel('Year')
plt.ylabel('YoY Growth (%)')
plt.grid(True)
plt.show()